In [1]:
# ============================================================================
# ZOMATO DELIVERY TIME PREDICTION - PHASE 1: DATA PREPARATION
# ============================================================================

import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# STEP 1: LOAD THE DATA
# ============================================================================

def load_data(file_path='Zomato_Dataset.csv'):
    """
    Load the dataset and display basic information
    """
    print(" Loading dataset...")
    df = pd.read_csv(file_path)
    
    print(f"\n Dataset Shape: {df.shape}")
    print(f" Columns: {list(df.columns)}")
    print(f"\n First 5 rows:")
    print(df.head())
    
    print(f"\n Dataset Info:")
    print(df.info())
    
    print(f"\n Basic Statistics:")
    print(df.describe(include='all'))
    
    return df

# ============================================================================
# STEP 2: CLEAN DATE/TIME COLUMNS
# ============================================================================

def clean_datetime_columns(df):
    """
    Clean and standardize all date/time columns
    """
    print("\n" + "="*60)
    print(" STEP 2: CLEANING DATE/TIME COLUMNS")
    print("="*60)
    
    # Create a copy to avoid SettingWithCopyWarning
    df_clean = df.copy()
    
    # ------------------------------------------------------------------------
    # 2.1: Fix inconsistent date formats in 'Order_Date'
    # ------------------------------------------------------------------------
    print("\n🔧 Cleaning 'Order_Date' column...")
    
    def parse_date(date_str):
        """Handle multiple date formats"""
        try:
            # Try format: DD/MM/YYYY or D/M/YYYY
            if '/' in str(date_str):
                return datetime.strptime(str(date_str), '%d/%m/%Y')
            # Try format: DD-MM-YYYY
            elif '-' in str(date_str):
                return datetime.strptime(str(date_str), '%d-%m-%Y')
        except:
            return pd.NaT
    
    df_clean['Order_Date'] = df_clean['Order_Date'].apply(parse_date)
    
    print(f" Converted to datetime. Sample: {df_clean['Order_Date'].iloc[0]}")
    
    # ------------------------------------------------------------------------
    # 2.2: Combine date with time columns
    # ------------------------------------------------------------------------
    print("\n🔧 Creating datetime columns...")
    
    # Create order datetime
    df_clean['Order_DateTime'] = pd.to_datetime(
        df_clean['Order_Date'].astype(str) + ' ' + df_clean['Time_Orderd'],
        errors='coerce'
    )
    
    # Create pickup datetime
    df_clean['Pickup_DateTime'] = pd.to_datetime(
        df_clean['Order_Date'].astype(str) + ' ' + df_clean['Time_Order_picked'],
        errors='coerce'
    )
    
    # ------------------------------------------------------------------------
    # 2.3: Calculate preparation time
    # ------------------------------------------------------------------------
    df_clean['Preparation_Time_Min'] = (
        df_clean['Pickup_DateTime'] - df_clean['Order_DateTime']
    ).dt.total_seconds() / 60
    
    # Handle negative preparation times (if any)
    df_clean['Preparation_Time_Min'] = df_clean['Preparation_Time_Min'].clip(lower=0)
    
    print(f" Created datetime columns.")
    print(f"   Preparation time stats:")
    print(f"   Min: {df_clean['Preparation_Time_Min'].min():.1f} min")
    print(f"   Max: {df_clean['Preparation_Time_Min'].max():.1f} min")
    print(f"   Avg: {df_clean['Preparation_Time_Min'].mean():.1f} min")
    
    # ------------------------------------------------------------------------
    # 2.4: Extract temporal features
    # ------------------------------------------------------------------------
    print("\n🔧 Extracting temporal features...")
    
    # Time of day features
    df_clean['Order_Hour'] = df_clean['Order_DateTime'].dt.hour
    df_clean['Order_Minute'] = df_clean['Order_DateTime'].dt.minute
    
    # Day features
    df_clean['Order_Day'] = df_clean['Order_DateTime'].dt.day
    df_clean['Order_Month'] = df_clean['Order_DateTime'].dt.month
    df_clean['Order_Year'] = df_clean['Order_DateTime'].dt.year
    df_clean['Order_DayOfWeek'] = df_clean['Order_DateTime'].dt.dayofweek  # Monday=0
    df_clean['Order_DayOfWeek_Name'] = df_clean['Order_DateTime'].dt.day_name()
    
    # Time period categories
    df_clean['Time_Period'] = pd.cut(
        df_clean['Order_Hour'],
        bins=[0, 6, 12, 18, 24],
        labels=['Night', 'Morning', 'Afternoon', 'Evening'],
        include_lowest=True
    )
    
    # Weekend flag
    df_clean['Is_Weekend'] = df_clean['Order_DayOfWeek'].isin([5, 6]).astype(int)
    
    print(f" Extracted {df_clean['Order_Hour'].nunique()} unique hours")
    print(f"   Weekend orders: {df_clean['Is_Weekend'].sum()} / {len(df_clean)}")
    
    return df_clean

# ============================================================================
# STEP 3: CALCULATE HAVERSINE DISTANCE
# ============================================================================

def calculate_haversine_distance(df):
    """
    Calculate distance between restaurant and delivery location using Haversine formula
    """
    print("\n" + "="*60)
    print("STEP 3: CALCULATING HAVERSINE DISTANCE")
    print("="*60)
    
    df_dist = df.copy()
    
    def haversine(lat1, lon1, lat2, lon2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees)
        
        Returns distance in kilometers
        """
        # Convert decimal degrees to radians
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        
        # Haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        
        # Radius of earth in kilometers (mean radius = 6371 km)
        radius = 6371.0
        
        return radius * c
    
    # Apply Haversine formula to calculate distance
    print("📏 Calculating distances...")
    df_dist['Distance_KM'] = df_dist.apply(
        lambda row: haversine(
            row['Restaurant_latitude'],
            row['Restaurant_longitude'],
            row['Delivery_location_latitude'],
            row['Delivery_location_longitude']
        ),
        axis=1
    )
    
    # Also calculate Manhattan distance approximation (for urban areas)
    # This can be useful for cities with grid-like road networks
    df_dist['Lat_Diff_KM'] = abs(df_dist['Delivery_location_latitude'] - 
                                df_dist['Restaurant_latitude']) * 111.32
    df_dist['Lon_Diff_KM'] = abs(df_dist['Delivery_location_longitude'] - 
                                df_dist['Restaurant_longitude']) * 111.32 * \
                                np.cos(np.radians(df_dist['Restaurant_latitude']))
    
    df_dist['Manhattan_Distance_KM'] = df_dist['Lat_Diff_KM'] + df_dist['Lon_Diff_KM']
    
    # Distance categories
    df_dist['Distance_Category'] = pd.cut(
        df_dist['Distance_KM'],
        bins=[0, 2, 5, 10, 20, 100],
        labels=['Very Close (<2km)', 'Close (2-5km)', 'Medium (5-10km)', 
                'Far (10-20km)', 'Very Far (>20km)']
    )
    
    print(" Distance calculation complete!")
    print(f"\n Distance Statistics (in KM):")
    print(f"   Min Distance: {df_dist['Distance_KM'].min():.2f} km")
    print(f"   Max Distance: {df_dist['Distance_KM'].max():.2f} km")
    print(f"   Mean Distance: {df_dist['Distance_KM'].mean():.2f} km")
    print(f"   Median Distance: {df_dist['Distance_KM'].median():.2f} km")
    
    print(f"\n Distance Distribution:")
    print(df_dist['Distance_Category'].value_counts().sort_index())
    
    return df_dist

# ============================================================================
# STEP 4: HANDLE MISSING VALUES
# ============================================================================

def handle_missing_values(df):
    """
    Identify and handle missing values in the dataset
    """
    print("\n" + "="*60)
    print("STEP 4: HANDLING MISSING VALUES")
    print("="*60)
    
    df_clean = df.copy()
    
    # ------------------------------------------------------------------------
    # 4.1: Check for missing values
    # ------------------------------------------------------------------------
    print("\n Missing Values Before Cleaning:")
    missing_before = df_clean.isnull().sum()
    missing_percent = (missing_before / len(df_clean)) * 100
    
    missing_df = pd.DataFrame({
        'Missing_Count': missing_before,
        'Missing_Percent': missing_percent
    })
    
    # Only show columns with missing values
    missing_df = missing_df[missing_df['Missing_Count'] > 0]
    
    if len(missing_df) > 0:
        print(missing_df.sort_values('Missing_Percent', ascending=False))
    else:
        print(" No missing values found!")
    
    # ------------------------------------------------------------------------
    # 4.2: Handle specific columns with missing values
    # ------------------------------------------------------------------------
    if 'multiple_deliveries' in df_clean.columns:
        print(f"\n🔧 Handling 'multiple_deliveries' column...")
        # Fill NaN with 0 (assuming NaN means no multiple deliveries)
        df_clean['multiple_deliveries'] = df_clean['multiple_deliveries'].fillna(0)
        print(f"   Filled {missing_before.get('multiple_deliveries', 0)} missing values with 0")
    
    # Check for other potential missing values in key columns
    key_columns = ['Delivery_person_Ratings', 'Weather_conditions', 
                   'Road_traffic_density', 'Type_of_order', 'Type_of_vehicle']
    
    for col in key_columns:
        if col in df_clean.columns and df_clean[col].isnull().any():
            if df_clean[col].dtype == 'object':
                # For categorical, fill with mode
                mode_val = df_clean[col].mode()[0]
                df_clean[col] = df_clean[col].fillna(mode_val)
                print(f"   Filled missing '{col}' with mode: {mode_val}")
            else:
                # For numerical, fill with median
                median_val = df_clean[col].median()
                df_clean[col] = df_clean[col].fillna(median_val)
                print(f"   Filled missing '{col}' with median: {median_val:.2f}")
    
    # ------------------------------------------------------------------------
    # 4.3: Check for missing values after cleaning
    # ------------------------------------------------------------------------
    print("\n Missing Values After Cleaning:")
    missing_after = df_clean.isnull().sum().sum()
    
    if missing_after == 0:
        print("All missing values handled successfully!")
    else:
        print(f" Still have {missing_after} missing values")
        # Show remaining missing values
        remaining_missing = df_clean.isnull().sum()
        remaining_missing = remaining_missing[remaining_missing > 0]
        print(remaining_missing)
    
    return df_clean

# ============================================================================
# STEP 5: ENCODE CATEGORICAL VARIABLES
# ============================================================================

def encode_categorical_variables(df):
    """
    Encode categorical variables for machine learning
    """
    print("\n" + "="*60)
    print(" STEP 5: ENCODING CATEGORICAL VARIABLES")
    print("="*60)
    
    df_encoded = df.copy()
    
    # ------------------------------------------------------------------------
    # 5.1: Identify categorical columns
    # ------------------------------------------------------------------------
    categorical_cols = df_encoded.select_dtypes(include=['object']).columns.tolist()
    
    # Remove ID columns and datetime columns that we've already processed
    id_cols = ['ID', 'Delivery_person_ID']
    date_cols = ['Order_Date', 'Time_Orderd', 'Time_Order_picked', 
                 'Order_DateTime', 'Pickup_DateTime', 'Order_DayOfWeek_Name']
    
    categorical_cols = [col for col in categorical_cols 
                       if col not in id_cols + date_cols]
    
    print(f" Categorical columns to encode: {categorical_cols}")
    
    # ------------------------------------------------------------------------
    # 5.2: Create encoding dictionary for tracking
    # ------------------------------------------------------------------------
    encoding_info = {}
    
    # ------------------------------------------------------------------------
    # 5.3: Apply different encoding strategies
    # ------------------------------------------------------------------------
    
    # A) Ordinal Encoding for ordered categories
    ordinal_mappings = {
        'Road_traffic_density': {'Low': 0, 'Medium': 1, 'High': 2, 'Jam': 3},
        'Vehicle_condition': {0: 0, 1: 1, 2: 2},  # Already numeric, but ensure it's treated as categorical
        'multiple_deliveries': {0: 0, 1: 1, 2: 2, 3: 3}  # Ordered
    }
    
    for col, mapping in ordinal_mappings.items():
        if col in df_encoded.columns:
            print(f"\n🔢 Ordinal encoding '{col}': {mapping}")
            df_encoded[f'{col}_Encoded'] = df_encoded[col].map(mapping)
            encoding_info[col] = {'type': 'ordinal', 'mapping': mapping}
    
    # B) One-Hot Encoding for nominal categories
    nominal_cols = ['Weather_conditions', 'Type_of_order', 
                    'Type_of_vehicle', 'City', 'Festival', 'Time_Period']
    
    # Filter to only columns that exist in dataframe
    nominal_cols = [col for col in nominal_cols if col in df_encoded.columns]
    
    print(f"\n🎭 One-hot encoding for: {nominal_cols}")
    
    for col in nominal_cols:
        if col in df_encoded.columns:
            # Get dummies and prefix with column name
            dummies = pd.get_dummies(df_encoded[col], prefix=col, drop_first=True)
            
            # Add to dataframe
            df_encoded = pd.concat([df_encoded, dummies], axis=1)
            
            # Store encoding info
            categories = df_encoded[col].unique().tolist()
            encoding_info[col] = {'type': 'one-hot', 'categories': categories}
            
            print(f"   '{col}' → {len(categories)} categories")
    
    # ------------------------------------------------------------------------
    # 5.4: Create binary flags for important categories
    # ------------------------------------------------------------------------
    print("\n Creating binary flags...")
    
    # Festival binary flag
    if 'Festival' in df_encoded.columns:
        df_encoded['Is_Festival'] = (df_encoded['Festival'] == 'Yes').astype(int)
    
    # Bad weather flag
    if 'Weather_conditions' in df_encoded.columns:
        bad_weather = ['Fog', 'Stormy', 'Sandstorms', 'Windy']
        df_encoded['Bad_Weather'] = df_encoded['Weather_conditions'].isin(bad_weather).astype(int)
    
    # Rush hour flag
    df_encoded['Is_Rush_Hour'] = df_encoded['Order_Hour'].isin([8, 9, 12, 13, 17, 18, 19]).astype(int)
    
    # ------------------------------------------------------------------------
    # 5.5: Display encoding summary
    # ------------------------------------------------------------------------
    print("\n Encoding Summary:")
    for col, info in encoding_info.items():
        if info['type'] == 'ordinal':
            print(f"   {col}: Ordinal → {info['mapping']}")
        else:
            print(f"   {col}: One-Hot → {len(info['categories'])} categories")
    
    # Count new features created
    original_cols = len(df.columns)
    new_cols = len(df_encoded.columns)
    print(f"\n Created {new_cols - original_cols} new encoded features")
    print(f"   Total features: {new_cols}")
    
    return df_encoded, encoding_info

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """
    Main function to execute all data preparation steps
    """
    print(" STARTING ZOMATO DELIVERY DATA PREPARATION")
    print("="*60)
    
    # ------------------------------------------------------------------------
    # 0. Load the data (replace with your actual file path)
    # ------------------------------------------------------------------------
    # Assuming the data is saved as 'zomato_delivery_data.csv'
    # If using the sample data, you might need to create this file first
    
    # For demonstration, let's create a sample dataframe from your sample rows
    sample_data = {
        'ID': ['0xcdcd', '0xd987', '0x2784', '0xc8b6', '0xdb64', '0x3af3', '0x3aab', '0x689b', '0x6f67', '0xc9cf', '0x36b8'],
        'Delivery_person_ID': ['DEHRES17DEL01', 'KOCRES16DEL01', 'PUNERES13DEL03', 'LUDHRES15DEL02', 'KNPRES14DEL02', 'MUMRES15DEL03', 'MYSRES01DEL01', 'PUNERES20DEL01', 'HYDRES14DEL01', 'KOLRES15DEL03', 'PUNERES19DEL02'],
        'Delivery_person_Age': [36, 21, 23, 34, 24, 29, 35, 33, 34, 21, 25],
        'Delivery_person_Ratings': [4.2, 4.7, 4.7, 4.3, 4.7, 4.5, 4.0, 4.2, 4.9, 4.7, 4.1],
        'Restaurant_latitude': [30.327968, 10.003064, 18.56245, 30.899584, 26.463504, 19.176269, 12.311072, 18.592718, 17.426228, 22.552672, 18.563934],
        'Restaurant_longitude': [78.046106, 76.307589, 73.916619, 75.809346, 80.372929, 72.836721, 76.654878, 73.773572, 78.407495, 88.352885, 73.915367],
        'Delivery_location_latitude': [30.397968, 10.043064, 18.65245, 30.919584, 26.593504, 19.266269, 12.351072, 18.702718, 17.496228, 22.582672, 18.643935],
        'Delivery_location_longitude': [78.116106, 76.347589, 74.006619, 75.829346, 80.502929, 72.926721, 76.694878, 73.883572, 78.477495, 88.382885, 73.995367],
        'Order_Date': ['12/2/2022', '13-02-2022', '4/3/2022', '13-02-2022', '14-02-2022', '2/4/2022', '1/3/2022', '16-03-2022', '20-03-2022', '15-02-2022', '16-03-2022'],
        'Time_Orderd': ['21:55', '14:55', '17:30', '9:20', '19:50', '20:25', '14:55', '20:30', '20:40', '21:15', '20:20'],
        'Time_Order_picked': ['22:10', '15:05', '17:40', '9:30', '20:05', '20:35', '15:10', '20:40', '20:50', '21:30', '20:25'],
        'Weather_conditions': ['Fog', 'Stormy', 'Sandstorms', 'Sandstorms', 'Fog', 'Sandstorms', 'Windy', 'Sandstorms', 'Cloudy', 'Windy', 'Sandstorms'],
        'Road_traffic_density': ['Jam', 'High', 'Medium', 'Low', 'Jam', 'Jam', 'High', 'Jam', 'Jam', 'Jam', 'Jam'],
        'Vehicle_condition': [2, 1, 1, 0, 1, 2, 1, 2, 0, 0, 0],
        'Type_of_order': ['Snack', 'Meal', 'Drinks', 'Buffet', 'Snack', 'Buffet', 'Meal', 'Snack', 'Snack', 'Meal', 'Snack'],
        'Type_of_vehicle': ['motorcycle', 'motorcycle', 'scooter', 'motorcycle', 'scooter', 'electric_scooter', 'scooter', 'motorcycle', 'motorcycle', 'motorcycle', 'motorcycle'],
        'multiple_deliveries': [3, 1, 1, 0, 1, 1, 1, 1, np.nan, 1, 2],
        'Festival': ['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No'],
        'City': ['Metropolitian', 'Metropolitian', 'Metropolitian', 'Metropolitian', 'Metropolitian', 'Metropolitian', 'Metropolitian', 'Metropolitian', 'Metropolitian', 'Urban', 'Metropolitian'],
        'Time_taken (min)': [46, 23, 21, 20, 41, 20, 33, 40, 41, 15, 36]
    }
    
    df = pd.DataFrame(sample_data)
    print(" Using sample data (11 rows from your example)")
    
    # ------------------------------------------------------------------------
    # 1. Clean date/time columns
    # ------------------------------------------------------------------------
    df = clean_datetime_columns(df)
    
    # ------------------------------------------------------------------------
    # 2. Calculate Haversine distance
    # ------------------------------------------------------------------------
    df = calculate_haversine_distance(df)
    
    # ------------------------------------------------------------------------
    # 3. Handle missing values
    # ------------------------------------------------------------------------
    df = handle_missing_values(df)
    
    # ------------------------------------------------------------------------
    # 4. Encode categorical variables
    # ------------------------------------------------------------------------
    df, encoding_info = encode_categorical_variables(df)
    
    # ------------------------------------------------------------------------
    # 5. Display final prepared data
    # ------------------------------------------------------------------------
    print("\n" + "="*60)
    print("DATA PREPARATION COMPLETE")
    print("="*60)
    
    print(f"\n Final Dataset Shape: {df.shape}")
    print(f" Columns ({len(df.columns)} total):")
    
    # Group columns by type
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    datetime_cols = df.select_dtypes(include=['datetime64']).columns.tolist()
    
    print(f"\n    Numeric columns ({len(numeric_cols)}):")
    print(f"      {numeric_cols[:10]}")
    if len(numeric_cols) > 10:
        print(f"      ... and {len(numeric_cols) - 10} more")
    
    print(f"\n   Categorical columns ({len(categorical_cols)}):")
    print(f"      {categorical_cols}")
    
    print(f"\n   Datetime columns ({len(datetime_cols)}):")
    print(f"      {datetime_cols}")
    
    # ------------------------------------------------------------------------
    # 6. Save the prepared data
    # ------------------------------------------------------------------------
    output_file = 'zomato_delivery_prepared.csv'
    df.to_csv(output_file, index=False)
    print(f"\n Prepared data saved to: {output_file}")
    
    # ------------------------------------------------------------------------
    # 7. Show key features for modeling
    # ------------------------------------------------------------------------
    print("\n" + "="*60)
    print("KEY FEATURES READY FOR MODELING")
    print("="*60)
    
    # Show correlation with target variable
    if 'Time_taken (min)' in df.columns:
        print("\n Correlation with Target (Time_taken):")
        numeric_features = [col for col in numeric_cols 
                           if col != 'Time_taken (min)' and 
                           not any(x in col for x in ['latitude', 'longitude', 'Diff'])]
        
        correlations = {}
        for col in numeric_features:
            if col in df.columns:
                corr = df[col].corr(df['Time_taken (min)'])
                correlations[col] = corr
        
        # Sort by absolute correlation
        sorted_corr = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)
        
        print("\nTop correlated features:")
        for feature, corr in sorted_corr[:10]:
            print(f"   {feature:30} : {corr:+.3f}")
    
    return df

# ============================================================================
# RUN THE SCRIPT
# ============================================================================

if __name__ == "__main__":
    # Execute the data preparation pipeline
    prepared_df = main()
    
    # Display the first few rows of prepared data
    print("\n" + "="*60)
    print(" SAMPLE OF PREPARED DATA")
    print("="*60)
    
    # Select key columns to display
    key_columns = [
        'Distance_KM', 'Preparation_Time_Min', 'Order_Hour', 
        'Is_Weekend', 'Road_traffic_density_Encoded',
        'Bad_Weather', 'Is_Rush_Hour', 'Time_taken (min)'
    ]
    
    # Filter to only columns that exist
    display_cols = [col for col in key_columns if col in prepared_df.columns]
    
    if display_cols:
        print(prepared_df[display_cols].head())

 STARTING ZOMATO DELIVERY DATA PREPARATION
 Using sample data (11 rows from your example)

 STEP 2: CLEANING DATE/TIME COLUMNS

🔧 Cleaning 'Order_Date' column...
 Converted to datetime. Sample: 2022-02-12 00:00:00

🔧 Creating datetime columns...
 Created datetime columns.
   Preparation time stats:
   Min: 5.0 min
   Max: 15.0 min
   Avg: 11.4 min

🔧 Extracting temporal features...
 Extracted 6 unique hours
   Weekend orders: 5 / 11

STEP 3: CALCULATING HAVERSINE DISTANCE
📏 Calculating distances...
 Distance calculation complete!

 Distance Statistics (in KM):
   Min Distance: 2.93 km
   Max Distance: 19.40 km
   Mean Distance: 10.64 km
   Median Distance: 10.76 km

 Distance Distribution:
Distance_Category
Very Close (<2km)    0
Close (2-5km)        2
Medium (5-10km)      2
Far (10-20km)        7
Very Far (>20km)     0
Name: count, dtype: int64

STEP 4: HANDLING MISSING VALUES

 Missing Values Before Cleaning:
                     Missing_Count  Missing_Percent
multiple_deliveries    